In [ ]:
import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.get_logger().setLevel('ERROR')
tf.debugging.set_log_device_placement(False)
from tensorflow import keras
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout,Input,BatchNormalization,MaxPooling1D,concatenate
# from tensorflow.keras.utils import np_utils,multi_gpu_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import regularizers
from tensorflow.keras import backend as bek
from tensorflow.keras.models import load_model


import os
os.environ["CUDA_VISIBLE_DEVICES"]="6,7"
# Specify the GPUs you want to use
# visible_devices = ['GPU:0', 'GPU:1']  # Adjust indices as needed
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import logging
logging.disable(logging.WARNING)



In [ ]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import gc
import matplotlib.pyplot as plt


%matplotlib inline



In [ ]:
"""
data cell
"""
df1=pd.read_csv('/home/dp/data/Bit_old/Bit_BP_new/datapath/pair_end_merge_SYS_random_400_split_up.csv',
                    index_col = 0)

print(df1.shape)

In [ ]:
for i,item in enumerate(df1['SYS']):
    if(df1['time'].iloc[i]==2):
        continue
    elif(df1['time'].iloc[i]==1):
        temp=df1[(df1['Person No']==df1['Person No'].iloc[i])&(df1['time']==2)]['SYS']
        if(temp.empty):
            continue
        else:
            #print(df1['SYS'].iloc[i],temp.iloc[0])
            df1['SYS'].iloc[i]=temp.iloc[0]
    else:
        continue

df1.head()

In [ ]:
cond2 = df1['count']!=0
df1 = df1[cond2]

target = 'SYS'


In [ ]:
df1['SD_last']=df1['SYS_last']-df1['DIA_last']

In [ ]:
'''
model cell
'''
def base_model():
    in_s = Input((406, 2))
    model_s = Conv1D(filters=256, kernel_size=3,  name="block1_conv") (in_s)
    model_s = BatchNormalization(axis=-1,  name="block1_batchnorm")(model_s)
    model_s = Activation('relu', name="block1_act")(model_s)
    model_s = MaxPooling1D(pool_size=2, name="block1_pool")(model_s)
    
    model_s = Conv1D(filters=256, kernel_size=3, name="block2_conv") (model_s)
    model_s = BatchNormalization(axis=-1,  name="block2_batchnorm")(model_s)
    model_s = Activation('relu', name="block2_act")(model_s)
    model_s = MaxPooling1D(pool_size=2, name="block2_pool")(model_s)
    
    model_s = Conv1D(filters=512, kernel_size=3, name="block3_conv") (model_s)
    model_s = BatchNormalization(axis=-1,  name="block3_batchnorm")(model_s)
    model_s = Activation('relu', name="block3_act")(model_s)
    model_s = MaxPooling1D(pool_size=2, name="block3_pool")(model_s)
    
    model_s = Conv1D(filters=1024, kernel_size=3, name="block4_conv") (model_s)
    model_s = BatchNormalization(axis=-1,  name="block4_batchnorm")(model_s)
    model_s = Activation('relu', name="block4_act")(model_s)
    model_s = MaxPooling1D(pool_size=2, name="block4pool")(model_s)
    
    model_s = Conv1D(filters=2048, kernel_size=3, name="block5_conv") (model_s)
    model_s = BatchNormalization(axis=-1,  name="block5_batchnorm")(model_s)
    model_s = Activation('relu', name="block5_act")(model_s)
    model_s = MaxPooling1D(pool_size=2, name="block5_pool")(model_s)
    
    model_s = Flatten(name="flatten")(model_s)
    
    input1 = keras.layers.Input(shape=(2,), name= "dl_input")
    c = keras.layers.Concatenate(axis=-1)([model_s, input1])
    
    m = BatchNormalization(axis=-1, name = "batch_norm")(c)
    m = Dense(4096,activation='relu', name= 'FC1')(m)
    m = Dense(2048,activation='relu', name= 'FC2')(m)
    out = Dense(1,activation='linear', name="predictions")(m)
    
    model = Model(inputs=[in_s,input1], outputs=[out])

    # Compile the model
    adam=keras.optimizers.Adam(lr=0.00001, beta_1=0.9, beta_2=0.999)
    model.compile(loss='mean_squared_error', optimizer='adam')

    return model

In [ ]:
# base_model().summary()

In [ ]:
def test_step(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,17:423]
    X_train_s_last = train_temp.iloc[:,425:831]
    X_test_s = test_temp.iloc[:,17:423]
    X_test_s_last = test_temp.iloc[:,425:831]
    
    f1=['SYS']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
    # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshape test data
    X_test_r_s = np.zeros((len(X_test_s), 406, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_last
    
    f2=['SYS_last','SD_last']
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape test data
    X_test_r_f = test_temp[f2]
    
    #fit
    filepath="SYS_weights_best-Copy1.hdf5"
    checkpoint = ModelCheckpoint(filepath, monitor='loss', save_best_only=True,mode='min')
    callbacks_list = [checkpoint]
    # Create a MirroredStrategy.
    tf.debugging.set_log_device_placement(False)
    mirrored_strategy = tf.distribute.MirroredStrategy()
    print("Number of devices: {}".format(mirrored_strategy.num_replicas_in_sync)) 
    
    with mirrored_strategy.scope():
        # Define your model
        model = base_model()  
    
    
    Y_train = Y_train.reshape((Y_train.shape[0],))
    
    train_dataset = tf.data.Dataset.from_tensor_slices(((X_train_r_s, X_train_r_f), Y_train))
    #train_dataset = train_dataset.batch(3000)
    
    options = tf.data.Options()
    options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA
    train_dataset = train_dataset.with_options(options)
    
    # seting it, https://stackoverflow.com/questions/49922252/choosing-number-of-steps-per-epoch
    steps_per_Epoch = int( np.ceil(X_train_r_s.shape[0] / 3000) )
    history = model.fit(train_dataset.batch(batch_size = 3000).repeat(),  epochs = 750,  steps_per_epoch = steps_per_Epoch, callbacks=callbacks_list)
    
    plt.plot(history.history['loss'])
    plt.ylim(0,0.2)
    plt.show()
    
    test_pred = model.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['SYS_p']=test_pred1
    
    temp_a=0
    temp_b=0
    test_SYS_r=[]
    test_SYS_p=[]

    
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            test_SYS_r.append(test_temp['SYS'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['SYS_p'])
            test_SYS_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]

            
    #print(test_SYS_p)
    #print(test_SYS_r)
    del model
    bek.clear_session()
    gc.collect()
    return (test_SYS_p,test_SYS_r)

In [ ]:
def test_step1(train_temp,test_temp):
    X_train_s = train_temp.iloc[:,17:423]
    X_train_s_last = train_temp.iloc[:,425:831]
    X_test_s = test_temp.iloc[:,17:423]
    X_test_s_last = test_temp.iloc[:,425:831]
    
    f1=['SYS']
    Y_train = train_temp[f1]
    Y_train = scalert.transform(Y_train)
    
   # reshape train data
    X_train_r_s = np.zeros((len(X_train_s), 406, 2))
    X_train_r_s[:, :, 0] = X_train_s
    X_train_r_s[:, :, 1] = X_train_s_last

    # reshap validation data
    X_test_r_s = np.zeros((len(X_test_s), 406, 2))
    X_test_r_s[:, :, 0] = X_test_s
    X_test_r_s[:, :, 1] = X_test_s_last
    
    f2=['SYS_last',"SD_last"]
    # reshape train data
    X_train_r_f = train_temp[f2]

    # reshape validation data
    X_test_r_f = test_temp[f2]
    
     # Create a MirroredStrategy.
    tf.debugging.set_log_device_placement(False)
    mirrored_strategy = tf.distribute.MirroredStrategy()
    with mirrored_strategy.scope():
        # Define your model
        model1 = base_model()  

    model1.load_weights("SYS_weights_best-Copy1.hdf5")
                        
    test_pred = model1.predict([X_test_r_s,X_test_r_f])
    test_pred1 = scalert.inverse_transform(test_pred)
    test_temp['SYS_p']=test_pred1
    
    temp_a=0
    temp_b=0
    test_SYS_r=[]
    test_SYS_p=[]

    
    for i,item in enumerate(test_temp['Person No']):
        if((test_temp['Person No'].iloc[i]!=temp_a) | (test_temp['time'].iloc[i]!=temp_b)):
            
            #administrator mode
            test_SYS_r.append(test_temp['SYS'].iloc[i])
            temp = test_temp[(test_temp['Person No']==test_temp['Person No'].iloc[i])&(test_temp['time']==test_temp['time'].iloc[i])]
            t = np.median(temp['SYS_p'])
            test_SYS_p.append(t)
            temp_a = test_temp['Person No'].iloc[i]
            temp_b = test_temp['time'].iloc[i]

    
    del model1
    bek.clear_session()
    gc.collect()
    
    return (test_SYS_p,test_SYS_r)

In [ ]:
n=9

savedir_results = 'SYS/Personalized_pair_of_'
dir_name = f'{savedir_results}_P{n:01d}'
if os.path.exists(dir_name):
    dir_name = dir_name
else:
    os.makedirs(dir_name)
    
#testingNo_all = [1273,543,1002,342,245,149,1888,1373,1020,1775,1598, 1436,2095,2096,1722]
testingNo_all = [342,149]
for testingNo in testingNo_all:
    print('###########',testingNo,'###########')
    
    test_p=[]
    test_r=[]
    test_p_best=[]
    test_r_best=[]
    
    train = df1
    leftout_data = df1[(df1['count']==testingNo)&(df1['count_time']>(n))]
    leftout_data_index = leftout_data.index
    train = train.drop(leftout_data_index)
    
    t = train[target]
    t = t.values.reshape(-1, 1)
    scalert = preprocessing.MinMaxScaler().fit(t)
    
    test  = df1[(df1['count']==testingNo)&(df1['count_time']>=10)]        

    (temp_pred,temp_real) = test_step(train,test)
    (temp_pred_best,temp_real_best) = test_step1(train,test)

    test_p=[temp_pred[i:i + 2] for i in range(0, len(temp_pred), 2)]
    test_r=[temp_real[i:i + 2] for i in range(0, len(temp_real), 2)]

    test_p_best=[temp_pred_best[i:i + 2] for i in range(0, len(temp_pred_best), 2)]
    test_r_best=[temp_real_best[i:i + 2] for i in range(0, len(temp_real_best), 2)]
        
    d = {'SYS_real': test_r, 'SYS_pred': test_p}
    df_f = pd.DataFrame(data = d)

    ind =[]
    for j in range(2,len(df_f.index)+2):
        ind.append(j)

    df_f.index=ind

    name_temp =str(testingNo)+'-1000-256.csv'
    df_f.to_csv(dir_name + '/' + name_temp)


    d_best = {'SYS_real': test_r_best, 'SYS_pred': test_p_best}
    df_f_best = pd.DataFrame(data = d_best)

    ind =[]
    for j in range(2,len(df_f_best.index)+2):
        ind.append(j)

    df_f_best.index=ind

    name_temp_best =str(testingNo)+'-1000-256-best.csv'
    df_f_best.to_csv(dir_name + '/' + name_temp_best)
    
        